In [1]:
import numpy as np
import pysocialforce as psf
import random

DEBUG:[__init__.py:275              wrapper() ] matplotlib data path: /home/guoda/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:[__init__.py:275              wrapper() ] CONFIGDIR=/home/guoda/.config/matplotlib
DEBUG:[__init__.py:1445             <module>() ] interactive is False
DEBUG:[__init__.py:1446             <module>() ] platform is linux
DEBUG:[__init__.py:1447             <module>() ] loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', 'encodings.latin_1', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', '_distutils_hack', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', '_heapq',

Parameters:

In [2]:
# Define initial state of agents
n = 10

# Define goal position
goal_position = np.array([2, 17.5])
goal_position2 = np.array([-10, 7.5])
goal_position3 = np.array([10, 7.5])

goals = np.array([goal_position, goal_position2, goal_position3])
obstacles = [[-10, 0, 15, 15], [2.5, 10, 15, 15], [-5, -5, 7.5, 15], [-5, -5, 5, 0], [5, 5, 7.5, 15], [5, 5, 5, 0], [-5, 5, 0, 0]]

In [3]:
#No knowledge movement

def random_movement():
    return np.random.uniform(-1, 1, size=2)

def stand_still():
    return np.array([0, 0])

def follow_stream():
    #add later maybe
    pass

# Helper function: aligns velocity with goal
def to_nearest_goal(position, goal):
    direction = goal - position
    norm = np.linalg.norm(direction)
    if norm == 0:
        return np.array([0, 0])
    return direction / norm  # Unit vector towards the goal

#Knowledge movement
    #->initialize_velocity() function or version of it

#NOTE: check radius and nearby ppl knowing

In [4]:
# Helper function: aligns velocity with goal
def initialize_velocity(position, goal):
    direction = goal - position
    norm = np.linalg.norm(direction)
    if norm == 0:
        return np.array([0, 0])
    return direction / norm  # Unit vector towards the goal

# Initialize each agent
initial_state = np.array([])
agent = []
for i in range(n):
	goal = random.choice(goals)
	x = random.uniform(-5, 5)
	y = random.uniform(0, 15)
	vx = 0 #initialize_velocity(np.array([x, y]), goal)[0]
	vy = 0 #initialize_velocity(np.array([x, y]), goal)[1]
	agent.append([x, y, vx, vy, goal[0], goal[1]])

initial_state = np.array(agent)

# Initialize the simulator
sim = psf.Simulator(
	initial_state, groups=None, obstacles=obstacles, config_file="loes.toml"
)

# Counter for agents that have reached the goal
agents_reached_goal = 0
agents_already_counted = set()  # Set to track counted agents to avoid double counting since removing agents from the simulator is not yet working

# Function to check if an agent has reached the goal
def check_reached_goal(agent_position, y_goal=15, x_goal=5, x_goal2=-5):
	# Check if the agent has reached the goal
	if agent_position[1] > y_goal or agent_position[0] > x_goal or agent_position[0] < x_goal2:
		return True
		

environment_knowledge = True
disaster_knowledge = True
# Perform simulation steps and remove agents that reach the goal 
for step in range(50):
	sim.step(n=1)
	ped_states, group_states = sim.get_states()  # Access the current state of agents

	# Extract the relevant frame (assuming we need the latest frame, index 0 for instance)
	current_state = ped_states[-1]  # Use the last frame for the current state

	new_ped_states = []
	for i, agent_state in enumerate(current_state):
		agent_position = agent_state[:2]  # Ensure agent_position is an array of shape (2,)

		if check_reached_goal(agent_position):
			if i not in agents_already_counted:
				agents_reached_goal += 1
				agents_already_counted.add(i)
		else:
			new_ped_states.append(agent_state)

			if environment_knowledge and disaster_knowledge:
				agent_state[2:4] = to_nearest_goal(agent_state[:2], goals[0])
			elif environment_knowledge and not disaster_knowledge:
				agent_state[2:4] = stand_still()
			elif not environment_knowledge and disaster_knowledge:
				agent_state[2:4] = random_movement()
			elif not environment_knowledge and not disaster_knowledge:
				agent_state[2:4] = stand_still()


	# Update the simulator's state directly with agents that haven't reached the goal
	sim.ped_states = [np.array(new_ped_states)] if new_ped_states else []

	if len(new_ped_states) == 0:
		break  # Exit the loop if all agents have reached the goal

print(f"Number of agents that reached the goal: {agents_reached_goal}")

DEBUG:[byteflow.py:74             __init__() ] bytecode dump:
>          0	NOP(arg=None, lineno=101)
           2	LOAD_FAST(arg=0, lineno=101)
           4	LOAD_CONST(arg=1, lineno=101)
           6	LOAD_CONST(arg=1, lineno=101)
           8	BUILD_SLICE(arg=2, lineno=101)
          10	LOAD_CONST(arg=2, lineno=101)
          12	LOAD_CONST(arg=3, lineno=101)
          14	BUILD_SLICE(arg=2, lineno=101)
          16	BUILD_TUPLE(arg=2, lineno=101)
          18	BINARY_SUBSCR(arg=None, lineno=101)
          20	STORE_FAST(arg=1, lineno=101)
          22	LOAD_GLOBAL(arg=0, lineno=102)
          24	LOAD_METHOD(arg=1, lineno=102)
          26	LOAD_CONST(arg=4, lineno=102)
          28	LOAD_CONST(arg=5, lineno=102)
          30	MAKE_FUNCTION(arg=0, lineno=102)
          32	LOAD_FAST(arg=1, lineno=102)
          34	GET_ITER(arg=None, lineno=102)
          36	CALL_FUNCTION(arg=1, lineno=102)
          38	CALL_METHOD(arg=1, lineno=102)
          40	STORE_FAST(arg=2, lineno=102)
          42	LOAD_FAST

Number of agents that reached the goal: 0


In [5]:
# Visualize the scene
with psf.plot.SceneVisualizer(sim, "mult_goals") as sv:
    sv.animate()

DEBUG:[pyplot.py:301       switch_backend() ] Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:[pyplot.py:301       switch_backend() ] Loaded backend module://matplotlib_inline.backend_inline version unknown.


DEBUG:[font_manager.py:1329     _findfont_cached() ] findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:[font_manager.py:1341     _findfont_cached() ] findfont: score(FontEntry(fname='/home/guoda/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono.ttf', name='DejaVu Sans Mono', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:[font_manager.py:1341     _findfont_cached() ] findfont: score(FontEntry(fname='/home/guoda/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizFourSymReg.ttf', name='STIXSizeFourSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:[font_manager.py:1341     _findfont_cached() ] findfont: score(FontEntry(fname='/home/guoda/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/cmr10.ttf', name='cmr10', style='normal', variant='normal', weight=400, stre